- Finding the best optimal resizing technique which can be applied to all Images.

I am seeing a lot of kernels where people are just resizing it to (64,64) to (128,128) without taking care of aspect ratio. So this notebook address this issue and suggest you methods to resize. 

- Aspect Ratio = width/height. 
 -if you are not maintaining the same aspect ratio when you are resizing the image. It will lead to a distorted Image and all the features which you r training using CNN's will fail in capturing signals as same kind of fish in different Images will have different shape because of your resizing technique. This is a serious false.
  

## Load the Libraries

In [ ]:
import numpy as np
import pandas as pd 
import glob,cv2
import random
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
from PIL import Image

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# load all the folder location
folderlist = glob.glob("../input/train/*")

In [ ]:
# Find what all different size Images you have - copied from other kaggle script
different_size = {}
for i in folderlist:
    filelist = glob.glob(i+"/*")
    for j in filelist:
        img = cv2.imread(j)
        size = "_".join(map(str,list(img.shape)))
        different_size[size] = different_size.get(size, 0) + 1

In [ ]:
different_size

In [ ]:
# select all the Image locations of the same shape and and append it to the list in a dict 
filelist_diff = {}
for i in tqdm(folderlist):
    filelist = glob.glob(i+"/*")
    for j in filelist:
        img = cv2.imread(j)
        size = "_".join(map(str,list(img.shape)))
        if size in filelist_diff.keys():
            filelist_diff[size].append(j)
        else:
            filelist_diff[size] = []
        

In [ ]:
#example
[random.choice(filelist_diff['700_1244_3']) for i in range(4)]

## Plots
 - Plot 2 Images each from same shape. I have randomly selected 2 Images and plotted it, You may get a different output when you run 

In [ ]:
for key in filelist_diff.keys():
    img_list = [random.choice(filelist_diff[key]) for i in range(4)]
    images = [cv2.imread(i) for i in img_list]
    fig,ax = plt.subplots(nrows=1,ncols=4,sharex="col",sharey="row",figsize=(20,3))
    fig.suptitle(key,fontsize=16)
    for i,img in enumerate(images):
        ax[i].imshow(img)

## Resizing - Here I  am resizing all the Images to (920,1200) size. 
- I have randomly selected 1200 to be my width as most of the Images have width around this value
- I have calculated the aspect ratio and as you can see they range from 1.31 to 1.77
- Next I have calculated the height of the Image incase if reduce/increase the width of the Image. This ranged from 913 to 674.
- So incase if we can create a board of (920,1200) and paste all the Images with center, We can bring all the Images to the same size. The for loop I have run where I used PIL package has done that for you. You can see the output.

In [ ]:
filelist_diff.keys()

In [ ]:
[float(i.rsplit("_")[1])/float(i.rsplit("_")[0]) for i in filelist_diff.keys()]

In [ ]:
[1200/(float(i.rsplit("_")[1])/float(i.rsplit("_")[0])) for i in filelist_diff.keys()]

In [ ]:
h = 1200
for key in filelist_diff.keys():
    img_list = [random.choice(filelist_diff[key]) for i in range(4)]
    Images = []
    for i in range(len(img_list)):
        img = Image.open(img_list[i],'r')
        l = int(img.size[1]*h/img.size[0])
        img = img.resize((h,l), Image.ANTIALIAS)
        background = Image.new('RGB', (1200, 920), (255, 255, 255))
        img_w, img_h = img.size
        bg_w, bg_h = background.size
        offset = (int((bg_w - img_w) / 2), int((bg_h - img_h) / 2))
        background.paste(img, offset)
        Images.append(background)
    fig,ax = plt.subplots(nrows=1,ncols=4,sharex="col",sharey="row",figsize=(20,3))
    fig.suptitle(key,fontsize=16)
    for i,f in enumerate(Images):
        ax[i].imshow(f)

Now using standard python libraries opencv and PIL, you can resize all the Images to the required size by preserving the aspect ratio and run your analysis 

I prefer using rectangular Images as input to CNN's as all train data have rectangular Images. 

Further, You can use the above functions and design a ImageGenerator function and feed it to your CNN's along with some data agumentation. 